# Projet Traitement Large Échelle

Pour ce projet j'ai choisi d'utiliser RQLite comme base de données relationelle, et KeyDB comme base de données NoSQL.

Nous allons tout d'abord étudier KeyDB, en détaillant son fonctionnement, puis en effectuant des tests de performance.

Ensuite, nous ferons de même pour RQLite, et finalement nous comparerons les deux bases de données.

## Dataset

Nous allons utiliser le dataset : https://opendata.paris.fr/explore/dataset/stationnement-sur-voie-publique-stationnement-interdit/

Il contient les emplacements de stationnement interdit dans la ville de Paris.
Ci-dessous un exemple de donnée.

In [2]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/40/10/79e52ef01dfeb1c1ca47a109a01a248754ebe990e159a844ece12914de83/pandas-2.2.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for numpy>=1.26.0 from https://files.pythonhosted.org/packages/0f/50/de23fde84e45f5c4fda2488c759b69990fd4512387a8632860f3ac9cd225/numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada

In [3]:
import pandas as pd

file_path = 'data/stationnement-sur-voie-publique-stationnement-interdit.json'
df = pd.read_json(file_path)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22533 entries, 0 to 22532
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      11126 non-null  float64
 1   id_old                  11407 non-null  object 
 2   regpri                  22533 non-null  object 
 3   regpar                  22533 non-null  object 
 4   arrond                  21021 non-null  float64
 5   zoneres                 21021 non-null  object 
 6   tar                     21021 non-null  object 
 7   typevoie                22533 non-null  object 
 8   nomvoie                 22533 non-null  object 
 9   parite                  22517 non-null  object 
 10  lon                     22533 non-null  float64
 11  longueur_calculee       22533 non-null  float64
 12  signhor                 22533 non-null  object 
 13  signvert                22533 non-null  object 
 14  confsign                22533 non-null

## Présentation de KeyDB

KeyDB est une version améliorée de Redis axée sur le multithreading, l'efficacité mémoire et le débit élevé.
Il conserve une compatibilité totale avec le protocole, les modules et les scripts Redis, tout en offrant des améliorations de performance telles que la réplication active et le stockage FLASH.

Grâce à son architecture MVCC, KeyDB permet l'exécution de requêtes telles que KEYS et SCAN sans bloquer la base de données ni dégrader les performances.
En utilisant le même matériel, KeyDB peut atteindre un débit significativement plus élevé que Redis.
 
Son architecture multithread simplifie la répartition des charges et permet une utilisation plus efficace des ressources matérielles. 
En outre, KeyDB offre une compatibilité avec les derniers développements de Redis, ce qui en fait un substitut direct pour les déploiements existants.

Sachant que KeyDB est un fork de redis, et qu'il est régulièrement synchronizé avec, il reste compatible avec tout les modules/extensions de Redis, nous parlerons donc de KeyDB/Redis.

KeyDB est généralement utilisé comme cache en raison de sa vitesse. Cependant, il peut faire bien plus grâce à ses modules. Ces extensions ajoutent des fonctionnalités comme la recherche en texte intégral, le traitement de données géospatiales, un moteur de recherche. Ainsi, bien qu'il soit souvent vu comme un simple cache, KeyDB peut s'adapter à une variété de cas d'utilisation grâce à ses modules, devenant ainsi une solution polyvalente pour divers besoins d'application.

Quelques avantages de KeyDB:

- Performances élevées : KeyDB offre des performances exceptionnelles grâce à son architecture multithreadée
- Compatibilité avec Redis : La compatibilité avec Redis permet aux utilisateurs de migrer facilement vers KeyDB sans nécessiter de modifications majeures du code
- Support de la communauté et des entreprises : Le projet est plutôt récent, mais bénéficie d'un soutien actif de la part de la communauté open-source ainsi que de certaines entreprises, assurant un développement continu et un support fiable.

Mais également quelques inconvénients

- Fonctionnalités limitées : Comparé à certains systèmes NoSQL plus établis, KeyDB peut manquer de certaines fonctionnalités avancées ou spécialisées, ce qui peut limiter son utilisation dans certains cas d'utilisation spécifiques.
- Documentation moins complète : Par rapport à des systèmes NoSQL plus populaires et matures, la documentation et les ressources d'apprentissage disponibles pour KeyDB peuvent être moins abondantes, ce qui peut poser des défis pour les nouveaux utilisateurs.
- Maturité relative : En tant que projet plutôt récent, KeyDB peut encore manquer de la stabilité et de la maturité des systèmes NoSQL plus établis, ce qui peut entraîner des problèmes potentiels de fiabilité ou de compatibilité.
- Dépendance aux threads : Bien que l'architecture multithreadée soit un avantage pour les performances, elle peut également introduire des complexités supplémentaires en matière de gestion des threads et de la concurrence dans le code, ce qui peut être un inconvénient pour certains développeurs.

### Installation et démarrage

Il est nécessaire d'avoir docker installé sur votre machine.

In [4]:
!docker run -p 6379:6379 --rm --name keydb-server -d eqalpha/keydb keydb-server /etc/keydb/keydb.conf --server-threads 4

5c50805ffcd491c777c132724c001e6f307b2738ca869bd355f9097531156616


Nous allons commencer par faire des tests de performance pour des opérations de CRUD.

Pour celà, il suffit d'installer la librairie redis, puis de se connecter au serveur KeyDB.

In [5]:
!pip install redis

  Obtaining dependency information for redis from https://files.pythonhosted.org/packages/65/f2/540ad07910732733138beb192991c67c69e7f6ebf549ce1a3a77846cbae7/redis-5.0.4-py3-none-any.whl.metadata
  Using cached redis-5.0.4-py3-none-any.whl.metadata (9.3 kB)
Using cached redis-5.0.4-py3-none-any.whl (251 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
import redis
import os
import platform
import time
import datetime
import math

In [7]:
# Connect to KeyDB
keydb_host = 'localhost'
keydb_port = 6379  # Default KeyDB port

# Create a connection to KeyDB
r = redis.Redis(host=keydb_host, port=keydb_port)

### Tests de performances

Toutes les mesures / serveurs tournent en local sur un PC qui contient:

In [8]:
system_info = platform.uname()

print("System Information:")
print(f"System: {system_info.system}")
print(f"Release: {system_info.release}")
print(f"Version: {system_info.version}")
print(f"Machine: {system_info.machine}")

System Information:
System: Linux
Release: 6.5.13-1-MANJARO
Version: #1 SMP PREEMPT_DYNAMIC Tue Nov 28 20:33:05 UTC 2023
Machine: x86_64


In [9]:
!grep -m 1 'model name' /proc/cpuinfo

model name	: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz


In [10]:
first = df.head(1)

first_id_old = first.id_old.iloc[0]
print(first_id_old)

530001D20131119153127


#### Insertion d'un champ

In [11]:
structure = str(first.iloc[0])

start_time = time.time()
r.set(first_id_old, str(first.to_json()))
end_time = time.time()

time_taken_insertion = end_time - start_time
print("insertion time:", str(datetime.timedelta(seconds=time_taken_insertion)))

insertion time: 0:00:00.002359


#### Lecture d'un champ

In [12]:
start_time = time.time()
found = r.get(first_id_old)
end_time = time.time()

if found is not None:
    time_taken_read = end_time - start_time
    print("read time:", str(datetime.timedelta(seconds=time_taken_read)))
else:
    print("no row found")

read time: 0:00:00.000524


#### Mise à jour d'un champ déjà existant

KeyDB est une base de données clés/valeurs, donc il n'y a pas de "mise à jour" proprement dite, mais juste une réécriture.
Cependant, KeyDB/Redis possède un type de données hashset, qui permet de stocker une sorte de hashmap dans une clé, c'est ce qu'on pourra benchmark et comparer par la suite.

In [13]:
structure = str(first.iloc[0])

start_time = time.time()
r.set(first_id_old, str(first.to_json()) * 2)
end_time = time.time()

time_taken_update = end_time - start_time
print("update time:", str(datetime.timedelta(seconds=time_taken_update)))

update time: 0:00:00.000825


#### Suppression d'un champ

In [14]:
start_time = time.time()
deleted_ct = r.delete(first_id_old)
end_time = time.time()

if deleted_ct == 1:
    time_taken_deletion = end_time - start_time
    print("deletion time:", str(datetime.timedelta(seconds=time_taken_deletion)))
else:
    print("no row deleted")

deletion time: 0:00:00.000587


#### Quelques observations

On remarque que les temps sont extrêmement réduits, et que tout se passe en microsecondes.

#### Insertion du dataset complet

Il existe deux techniques, insertions l'une après l'autre des données ou l'insertion en masse.

L'insertion en masse est une technique nécessitant de suivre un protocole dédié à KeyDB.
Elle permet une insertion très rapide de beaucoup de données, car chaque SET unique est une opération à part entière et devient très vite, en cas d'un grand nombre de clés, une solution très lente et à éviter.

Nous n'allons pas étudier les résultats de l'insertion en masse, mais je vais vous présenter comment cela fonctionne.

Le protocole qu'il faut suivre se présente comme ci-dessous, prenons par exemple la commande <b>SET key value</b> :

```
*3<cr><lf>     <- On détermine d'abord le nombre d'arguments
$3<cr><lf>     <- Longueur de la commande
SET<cr><lf>    <- Commande SET
$3<cr><lf>     <- Longueur de la clé
key<cr><lf>    <- Valeur de la clé 
$5<cr><lf>     <- Longueur de valeur
value<cr><lf>  <- Valeur
```

Ainsi sur une ligne celà ressemblerait à :

`"*3\r\n$3\r\nSET\r\n$3\r\nkey\r\n$5\r\nvalue\r\n"`

Il suffit donc de faire un script qui transforme les données d'entrée en suivant ce protocole, puis :

`$ python conv.py data/stationnement-sur-voie-publique-stationnement-interdit.json | keydb-cli --pipe`

#### Insertion séquentielle complète du dataset

In [15]:
start_time = time.time()
total = 0
skipped = 0

for index, row in df.iterrows():
    id_old = row['id_old']
    if id_old is None:
        id_old = row['id']
    if id_old is None:
        skipped += 1
        continue
    structure = str(row.to_json())
    r.set(id_old, structure)
    total += 1

end_time = time.time()

time_taken_total_insertion = end_time - start_time
print("total dataset insertion time (success: %d/%d)" % (total, total+skipped), str(datetime.timedelta(seconds=time_taken_total_insertion)))

total dataset insertion time (success: 22533/22533) 0:00:06.041676


In [16]:
def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

In [17]:
convert_size(os.path.getsize("data/stationnement-sur-voie-publique-stationnement-interdit.json"))

'25.07 MB'

On remarque que pour un dataset contenant <b>22533</b> clés et d'une grandeur totale de <b>25 MB</b>, on prend 6 secondes.

### À propos du partitionnement

Le partitionnement dans KeyDB divise les données entre plusieurs instances pour améliorer les performances et la capacité de stockage.
Pour se faire, deux méthodes principales sont utilisées : 

1) Le partitionnement par plage: il consiste à mapper des plages d'objets sur des instances spécifiques de KeyDB. Par exemple, on peut décider que les utilisateurs avec des identifiants de 0 à 10000 seront stockés dans l'instance R0, tandis que les utilisateurs avec des identifiants de 10001 à 20000 seront stockés dans l'instance R1, et ainsi de suite. Bien que ce système soit simple, il nécessite une table de correspondance des plages vers les instances, ce qui peut être inefficace à grande échelle.

2) Le partitionnement par hachage: il fonctionne avec n'importe quelle clé et est réalisé en deux étapes simples : d'abord, on applique une fonction de hachage à la clé pour obtenir un nombre, puis on effectue une opération modulo pour mapper ce nombre sur une instance spécifique de KeyDB. Par exemple, si la clé est "foobar" et que le résultat du hachage modulo 4 est 2, alors la clé "foobar" sera stockée dans l'instance R2 de KeyDB. Ce système est plus flexible et ne nécessite pas de table de correspondance.

## Présentation de RQLite

RQLite est une base de données relationnelle qui combine la simplicité de SQLite avec la fiabilité d'un système distribué robuste. Il est facile à déployer et à utiliser. Basé sur SQLite, il assure le stockage fiable des données tout en offrant des fonctionnalités distribuées.

Quelques avantages:

- Léger et intégrable: rqlite est une base de données légère qui peut être facilement intégrée aux applications existantes.
- Distribué et tolérant aux pannes: rqlite réplique les données sur plusieurs nœuds, ce qui permet de maintenir la disponibilité des données même en cas de défaillance d'un nœud.
- Durable: rqlite prend en charge les transactions ACID, ce qui garantit l'intégrité des données.
- Scalable: rqlite peut être facilement mise à l'échelle horizontalement en ajoutant des nœuds supplémentaires au cluster.